bluecast处理完数据后，加载模型训练的功能学习

处理完数据后，对数据drop掉一些，设置模型的参数，自定义myCustomPrepocessing

In [1]:
import pandas as pd 
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [2]:
target = "NObeyesdad"

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [4]:
import numpy as np
def transform_feats(data):
    data['Gender_binary'] = data['Gender'].map({'Male': 1, 'Female': 0}).astype(int)
    data['family_history_with_overweight_binary'] = data['family_history_with_overweight'].map({'yes': 1, 'no': 0}).astype(int)
    data['SMOKE_binary'] = data['SMOKE'].map({'yes': 1, 'no': 0}).astype(int)
    
    # taken from: https://www.kaggle.com/code/ravi20076/playgrounds4e02-extraftre-models
    data["CAEC"] = data["CAEC"].map({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3}).astype(np.uint8)
    data['SCC']  = np.where(data["SCC"] == "no", 1,0).astype(np.uint8)
    data["CALC"] = data["CALC"].map({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 2}).astype(np.uint8)
    return data

def feature_engineering(data):
    # BMI
    data['BMI'] = data['Weight'] / (data['Height'] ** 2)
    # Activity
    data['Activity'] = data['FAF'] * data['TUE']
    # Age group
    data['Age_Group'] = pd.cut(data['Age'], bins=[0, 18, 30, 45, float('inf')], labels=[0, 1, 2, 3])
    data['Age_Group'] = data['Age_Group'].astype(int)
    # Height group
    data['Height_Group'] = pd.cut(data['Height'], bins=[0, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, float('inf')], labels=[0, 1, 2, 3, 4, 5, 6])
    data['Height_Group'] = data['Height_Group'].astype(int)
    #Risk score
    data['Risk factor'] = (data['BMI'] + data['Age_Group']) * (data["family_history_with_overweight_binary"] + data["SMOKE_binary"])
    
    # taken from: https://www.kaggle.com/code/ravi20076/playgrounds4e02-extraftre-models
    data["BMIbyNCP"] = np.log1p(data["BMI"]) - np.log1p(data["NCP"])
    data["BMIFAF"] = (data["BMI"] * data["FAF"])/ 25.0
    data["FAFmTUE"] = data["FAF"] - data["TUE"]
    data["FCVCpNCP"] = data['FCVC'] * data['NCP']
    data['TechUse'] = np.log1p(data['TUE']) - np.log1p(data['Age'])
    return data

In [5]:
train=transform_feats(train)
test=transform_feats(test)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     20758 non-null  int64  
 1   Gender                                 20758 non-null  object 
 2   Age                                    20758 non-null  float64
 3   Height                                 20758 non-null  float64
 4   Weight                                 20758 non-null  float64
 5   family_history_with_overweight         20758 non-null  object 
 6   FAVC                                   20758 non-null  object 
 7   FCVC                                   20758 non-null  float64
 8   NCP                                    20758 non-null  float64
 9   CAEC                                   20758 non-null  uint8  
 10  SMOKE                                  20758 non-null  object 
 11  CH

#示例数据
data = {
    'StudentID': [1, 2, 3, 4, 5, 6],
    'Class': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Subject': ['Math', 'Math', 'English', 'English', 'Science', 'Science'],
    'Score': [85, 90, 75, 80, 95, 85]
}

#创建DataFrame
df = pd.DataFrame(data)
现在，我们想要计算每个班级（Class）在每个科目（Subject）上的成绩的均值和标准差，然后计算z-scores。我们调用 get_group_zscores 函数：

#使用函数
df_with_zscores = get_group_zscores(df, ['Class', 'Subject'], 'Score')

In [7]:
def get_group_zscores(df, group_cols, agg_col):
    df_gr = df.groupby(group_cols).agg({agg_col: ["mean", "std"]}).droplevel(0, axis=1).reset_index()
    df_gr["mean"] = df_gr["mean"].fillna(df_gr["mean"].mean())
    df_gr["std"] = df_gr["std"].fillna(df_gr["std"].mean())
    
    identifier = "_".join(group_cols) + "_" + agg_col
    df_gr = df_gr.rename(
        columns = {
            "mean": f"{identifier}_mean",
            "std": f"{identifier}_std"
        }
    )
    df = df.merge(df_gr, on=group_cols, how="left")
    df[f"{identifier}_zscore"] = (df[f"{identifier}_mean"] - df[agg_col]) / df[f"{identifier}_std"]
    return df

In [13]:
# 示例数据
data = {
    'StudentID': [1, 2, 3, 4, 5, 6],
    'Class': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Subject': ['Math', 'Math', 'English', 'English', 'Science', 'Science'],
    'Score': [85, 90, 75, 80, 95, 85]
}

# 创建DataFrame
df = pd.DataFrame(data)
get_group_zscores(df, ['Class', 'Subject'], 'Score')

,StudentID,Class,Subject,Score,Class_Subject_Score_mean,Class_Subject_Score_std,Class_Subject_Score_zscore
0,1,A,Math,85,87.5,3.535534,0.707107
1,2,A,Math,90,87.5,3.535534,-0.707107
2,3,B,English,75,77.5,3.535534,0.707107
3,4,B,English,80,77.5,3.535534,-0.707107
4,5,C,Science,95,90.0,7.071068,-0.707107
5,6,C,Science,85,90.0,7.071068,0.707107


In [8]:
from bluecast.blueprints.cast import BlueCast
from bluecast.preprocessing.custom import CustomPreprocessing
from typing import Tuple,Optional
from sklearn.ensemble import IsolationForest


# add custom last mile computation
class CustomInFoldPreprocessing(CustomPreprocessing):
    def __init__(self):
        self.if_detector = None
    # Please note: The base class enforces that the fit_transform method is implemented
    def fit_transform(
        self, df: pd.DataFrame, target: pd.Series
    ) -> Tuple[pd.DataFrame, pd.Series]:
        train = df.copy()
        train = feature_engineering(train)
        train = get_group_zscores(train, ["Height_Group", "Age_Group"], "BMI")
        train = get_group_zscores(train, ["Height_Group", "Age_Group"], "Risk factor")
        train = get_group_zscores(train, ["Gender_binary", "SMOKE_binary", "family_history_with_overweight_binary"], "BMI")
        train = get_group_zscores(train, ["Gender_binary", "SMOKE_binary", "family_history_with_overweight_binary"], "Risk factor")
        train = train.drop(["source", "Age_Group", "Height_Group"], axis=1)
        train_target = target.copy().astype(float)
        train = train.replace([np.inf, -np.inf], 0)
        
        # add outlier scores as feature
        self.if_detector = IsolationForest(random_state=0)
        self.if_detector.fit(train.fillna(0))
        train["isolation_forest_scores"] = self.if_detector.predict(train.fillna(0))
        return train, train_target

    # Please note: The base class enforces that the fit_transform method is implemented
    def transform(
        self,
        df: pd.DataFrame,
        target: Optional[pd.Series] = None,
        predicton_mode: bool = False,
    ) -> Tuple[pd.DataFrame, Optional[pd.Series]]:
        train = df.copy()
        
        if isinstance(target, pd.Series) or isinstance(target, np.ndarray):
            train["target"] = target.copy()
            train["target"] = train["target"].astype(float)
            train = train.loc[train["source"] == 0].reset_index(drop=True) # no original data
            target = train.pop("target")
            target = pd.Series(target).astype(float)
        
        train = feature_engineering(train)
        train = get_group_zscores(train, ["Height_Group", "Age_Group"], "BMI")
        train = get_group_zscores(train, ["Height_Group", "Age_Group"], "Risk factor")
        train = get_group_zscores(train, ["Gender_binary", "SMOKE_binary", "family_history_with_overweight_binary"], "BMI")
        train = get_group_zscores(train, ["Gender_binary", "SMOKE_binary", "family_history_with_overweight_binary"], "Risk factor")
        train = train.drop(["source", "Age_Group", "Height_Group"], axis=1)
        train = train.replace([np.inf, -np.inf], 0)
        
        train["isolation_forest_scores"] = self.if_detector.predict(train.fillna(0))
        return train, target

custom_preprocessor = CustomInFoldPreprocessing()

d:\anaconda3\envs\bluecast\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_verbosity" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\anaconda3\envs\bluecast\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_verbosity_during_final_training" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\anaconda3\envs\bluecast\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_objective" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\anaconda3\envs\bluecast\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_eval_metric" has conflict with prot

In [9]:
from bluecast.config.training_config import TrainingConfig, XgboostTuneParamsConfig
from sklearn.model_selection import RepeatedStratifiedKFold
# We give more depth
xgboost_param_config = XgboostTuneParamsConfig()
xgboost_param_config.steps_max = 1000 #epochs数量：过多则浪费计算资源，过拟合风险；过少可能欠拟合
xgboost_param_config.max_depth_max = 7 #树的深度：过深有过拟合风险

# Create a custom training config and adjust general training parameters
train_config = TrainingConfig()
train_config.global_random_state = 600 #全局随机状态：保证实验可重复
train_config.hypertuning_cv_folds = 1 #交叉验证的分割数量：一般是5，增加计算成本，同时减少过拟合风险
train_config.hyperparameter_tuning_rounds = 500 #超参数调优的轮数：500意味着将尝试500种不同的超参数组合
train_config.hyperparameter_tuning_max_runtime_secs = 60 * 60 * 2 #最大运行时间为2小时
#train_config.enable_grid_search_fine_tuning = True #启用精细调优，模型将在更细致的参数网格上进行搜索
train_config.use_full_data_for_final_model = True 
#使用全部数据训练：最终模型可能会增加过拟合的风险，因为模型可能会过度学习训练数据中的噪声。
#如果设置为False，模型可能在一定程度上避免过拟合。
#如果最终模型只在部分数据上训练，那么模型的稳定性可能会受到影响，因为它可能没有充分学习到数据的整体分布。
train_config.precise_cv_tuning = True #启用精确的交叉验证调优可能意味着在调优过程中会更加精确地评估模型性能。
train_config.gridsearch_nb_parameters_per_grid = 5 #在搜索最佳超参数组合时，每个超参数要尝试多少个不同的值。
#train_config.cat_encoding_via_ml_algorithm = True 
#train_config.calculate_shap_values = False

skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1987)

In [16]:
from bluecast.blueprints.cast_cv import BlueCastCV
automl = BlueCastCV(
        class_problem="multiclass", # also multiclass is possible
        #stratifier=skf,
        conf_training=train_config,
        conf_xgboost=xgboost_param_config,
        custom_in_fold_preprocessor=custom_preprocessor,
        #custom_preprocessor=custom_preprocessor,
        #ml_model=custom_model_tab,
        )

In [17]:
try:
    automl.fit_eval(train.copy(), target_col=target)
except Exception as e:
    print(e)

d:\anaconda3\envs\bluecast\lib\site-packages\bluecast\blueprints\cast_cv.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, target_col] = y_train


Start fitting model number 0 with random seed 633
2024-03-10 07:13:37.426874: Start checking if GPU is available for usage.


d:\anaconda3\envs\bluecast\lib\site-packages\xgboost\core.py:160: UserWarning: [15:13:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\anaconda3\envs\bluecast\lib\site-packages\xgboost\core.py:160: UserWarning: [15:13:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
d:\anaconda3\envs\bluecast\lib\site-packages\bluecast\preprocessing\feature_types.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-

Invalid Input: 'cuda', valid values are: {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}
Xgboost uses CPU.
2024-03-10 07:13:37.672025: Start detecting and casting feature types.
2024-03-10 07:13:37.764045: Start fitting target label encoder.
2024-03-10 07:13:37.771049: Start encoding target labels.
2024-03-10 07:13:37.782050: Start executing train-test split with train size of 0.8.
2024-03-10 07:13:37.820058: Start filling infinite values.
2024-03-10 07:13:37.834022: Start filling infinite values.
2024-03-10 07:13:37.837469: Start date column conversion.
2024-03-10 07:13:37.859208: Start date column conversion.
2024-03-10 07:13:37.871365: Start fitting DataFrame schema.
2024-03-10 07:13:37.872285: Start checking if DataFrame schema of new data is consistent with previous data.
2024-03-10 07:13:37.875697: Start checking if DataFrame schema of new data is consistent with previous data.
2024-03-10 07:13:37.885698: Start fitting binary target encoder.
2024-03-10 07:13:38.012534: Start tran

d:\anaconda3\envs\bluecast\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
d:\anaconda3\envs\bluecast\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
d:\anaconda3\envs\bluecast\lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated a

2024-03-10 07:13:38.083552: Start transforming categories with multiclass target encoder.
2024-03-10 07:13:38.096554: Start fitting Xgboost model.
2024-03-10 07:13:38.096554: Start loading existing or default config files..
2024-03-10 07:13:38.096554: Found provided TrainingConfig.
2024-03-10 07:13:38.096554: Found provided XgboostTuneParamsConfig.
2024-03-10 07:13:38.096554: Found provided XgboostFinalParamConfig.
2024-03-10 07:13:38.127561: Start hyperparameter tuning of Xgboost model.
2024-03-10 07:13:38.128565: Start checking if GPU is available for usage.
Invalid Input: 'cuda', valid values are: {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}
Xgboost uses CPU.
2024-03-10 07:13:38.134562: Start loading existing or default config files..
2024-03-10 07:13:38.134562: Found provided TrainingConfig.
2024-03-10 07:13:38.134562: Found provided XgboostTuneParamsConfig.
2024-03-10 07:13:38.134562: Found provided XgboostFinalParamConfig.


Best trial: 4. Best value: -0.890916: 100%|██████████| 500/500 [19:10<00:00,  2.30s/it, 1150.01/7200 seconds] 


Best params: {'objective': 'multi:softprob', 'booster': 'gbtree', 'eval_metric': 'mlogloss', 'tree_method': 'exact', 'num_class': 7, 'max_depth': 3, 'alpha': 1.799630072849584, 'lambda': 6.311168399434964, 'gamma': 1.0599636288857384, 'max_leaves': 0, 'subsample': 0.770721262749801, 'colsample_bytree': 0.5794484910138618, 'colsample_bylevel': 0.9923315418891239, 'eta': 0.14935042200972282, 'steps': 667}
Finished hyperparameter tuning
Start final model training
"['source'] not found in axis"


In [19]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
for model in automl.bluecast_models:
    plot_tree(model.ml_model.model)
    fig = plt.gcf()
    fig.set_size_inches(150, 80)
    plt.show()

0
